<a href="https://colab.research.google.com/github/IlyaTsibulin/colab-tryon/blob/main/IDM_VTON(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/yisol/IDM-VTON.git
%cd IDM-VTON

Cloning into 'IDM-VTON'...
remote: Enumerating objects: 1468, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 1468 (delta 68), reused 58 (delta 58), pack-reused 1270
Receiving objects: 100% (1468/1468), 21.37 MiB | 9.86 MiB/s, done.
Resolving deltas: 100% (387/387), done.
/content/IDM-VTON


In [ ]:
!pip install torch torchvision pillow accelerate einops
!pip install diffusers

  Using cached fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
  Using cached psutil-6.0.0-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (21 kB)
  Using cached huggingface_hub-0.23.4-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 848.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.0 MB/s eta 0:00:

In [ ]:
!mkdir -p models/yisol/IDM-VTON
!wget -O models/yisol/IDM-VTON/diffusion_pytorch_model.bin https://huggingface.co/yisol/IDM-VTON/resolve/main/unet/diffusion_pytorch_model.bin

--2024-07-11 14:03:07--  https://huggingface.co/yisol/IDM-VTON/resolve/main/unet/diffusion_pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 54.192.18.15, 54.192.18.10, 54.192.18.113, ...
Connecting to huggingface.co (huggingface.co)|54.192.18.15|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/87/63/87637b3acda0183bf36504e203cfbe4cef0486cebd11a94d00e5bc19743ee69c/046b775cb9bbc67635fc3b148bb03bfe00496ce2f9ce8488a82fdb388669a521?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27diffusion_pytorch_model.bin%3B+filename%3D%22diffusion_pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1720964010&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMDk2NDAxMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzg3LzYzLzg3NjM3YjNhY2RhMDE4M2JmMzY1MDRlMjAzY2ZiZTRjZWYwNDg2Y2ViZDExYTk0ZDAwZTViYzE5NzQzZWU2OWMvMDQ

In [ ]:
!ls models/yisol/IDM-VTON/

diffusion_pytorch_model.bin


In [ ]:
!mkdir -p test/image
!mkdir -p test/cloth

In [ ]:
from google.colab import files
print("Upload test images")
uploaded = files.upload()
for filename in uploaded.keys():
    !mv {filename} test/image/

print("Upload clothing images")
uploaded = files.upload()
for filename in uploaded.keys():
    !mv {filename} test/cloth/

Upload test images


Saving barinow.jpg to barinow.jpg
Upload clothing images


Saving футболка.jpg to футболка.jpg


In [ ]:
print(f"Number of test images: {len(image_files)}")
print(f"Number of cloth images: {len(cloth_files)}")

Number of test images: 2
Number of cloth images: 2


In [ ]:
import os

image_files = os.listdir('/content/IDM-VTON/test/image')
cloth_files = os.listdir('/content/IDM-VTON/test/cloth')

if len(image_files) != len(cloth_files):
    print("Error: The number of image files and cloth files must be the same.")
else:
    with open('test/test_pairs.txt', 'w') as f:
        for image_file, cloth_file in zip(image_files, cloth_files):
            f.write(f"{image_file} {cloth_file}\n")

In [ ]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python3.8/site-packages')

--2024-07-11 15:07:22--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.32.241, 104.16.191.158, 2606:4700::6810:20f1, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.32.241|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146836934 (140M) [application/octet-stream]
Saving to: ‘Miniconda3-latest-Linux-x86_64.sh’

Miniconda3-latest-L 100%[===================>] 140.03M   202MB/s    in 0.7s    

2024-07-11 15:07:22 (202 MB/s) - ‘Miniconda3-latest-Linux-x86_64.sh’ saved [146836934/146836934]

PREFIX=/usr/local
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPA

In [ ]:
!conda env create -f environment.yaml
!source activate idm


CondaValueError: prefix already exists: /usr/local/envs/idm



In [ ]:
!python inference.py --width 563 --height 563 --num_inference_steps 30 --output_dir "result" --unpaired --data_dir "test" --seed 42 --test_batch_size 2 --guidance_scale 2.0

Traceback (most recent call last):
  File "/content/IDM-VTON/inference.py", line 15, in <module>
    from ip_adapter.ip_adapter import Resampler
  File "/content/IDM-VTON/ip_adapter/__init__.py", line 1, in <module>
    from .ip_adapter import IPAdapter, IPAdapterPlus, IPAdapterPlusXL, IPAdapterXL, IPAdapterFull,IPAdapterPlus_Lora,IPAdapterPlus_Lora_up
  File "/content/IDM-VTON/ip_adapter/ip_adapter.py", line 6, in <module>
    from diffusers.pipelines.controlnet import MultiControlNetModel
ImportError: cannot import name 'MultiControlNetModel' from 'diffusers.pipelines.controlnet' (/usr/local/lib/python3.12/site-packages/diffusers/pipelines/controlnet/__init__.py)
